### Arcface LFW AgeDB CFP-FP evaluation


In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training, extract_face
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler, SequentialSampler
from torchvision import datasets, transforms
import numpy as np
import os

In [2]:
# data_dir = 'D:/lfw'
# pairs_path = 'pairs.txt'
data_dir = 'D:\\AgeDB_data\\AgeDB'
pairs_path = 'D:\\faceVerificationData\\data\\AgeDB_pairs.txt'

In [3]:
# 打开文件  
with open('D:\\faceVerificationData\\data\\image_pathAgeDB.txt', 'r') as file:  
    # 读取所有行并存储到列表中  
    lines = file.readlines()  
  
# 去除每行末尾的换行符（如果有的话）  
crop = [line.strip() for line in lines]  
  
# 打印结果  
print(crop)

['D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0001.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0002.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0003.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0004.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0005.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0006.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0007.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0008.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0009.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0010.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0011.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0012.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0013.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0014.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0015.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0016.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_0017.jpg', 'D:\\AgeDB_data\\AgeDB\\AbeVigoda\\AbeVigoda_00

In [4]:
def list_txt(path, list=None):
    '''

    :param path: 储存list的位置
    :param list: list数据
    :return: None/relist 当仅有path参数输入时为读取模式将txt读取为list
             当path参数和list都有输入时为保存模式将list保存为txt
    '''
    if list != None:
        file = open(path, 'w')
        file.write(str(list))
        file.close()
        return None
    else:
        file = open(path, 'r')
        rdlist = eval(file.read())
        file.close()
        return rdlist


In [6]:
# crop=list_txt(path='D:\\faceVerificationData\\data\\crop_paths.txt')




In [5]:
def loadtxtmethod(filename):
    data = np.loadtxt(filename,dtype=np.float32,delimiter=' ')
    return data

In [6]:
#message=loadtxtmethod('embeddings.txt')
message=loadtxtmethod('D:\\faceVerificationData\\data\\embeddingsAgeDB.txt')

In [7]:
print(len(crop))
print(len(message))

16557
16557


In [8]:
print(type(message[0]))

<class 'numpy.ndarray'>


In [9]:
mess=message

In [10]:
print(mess)

[[-0.19722901  1.4794115   1.514209   ...  0.68148124 -0.30915388
  -0.28762692]
 [ 0.8110042   0.4261402   1.1972523  ...  0.3423461  -0.2237148
  -1.0955627 ]
 [ 0.02066806  1.8132843   1.0093315  ...  0.97316676 -1.8209513
   0.04822047]
 ...
 [ 1.6878786  -0.36986938 -0.291623   ... -0.7688866  -0.23188183
   0.22286697]
 [-1.3422089   0.38587153  0.37511122 ... -1.4931929   0.14186287
   2.1336303 ]
 [ 1.3106097  -0.5189201   0.19899347 ...  0.35538593 -0.18829477
  -0.26763454]]


In [11]:
# 计算每一行的模长  
row_lengths = np.linalg.norm(mess, axis=1)  
  
# 标准化每一行  
message = np.divide(message, row_lengths[:, np.newaxis])  
  
print(message)

[[-0.0087226   0.0654281   0.06696705 ...  0.03013903 -0.01367257
  -0.01272052]
 [ 0.04065377  0.02136142  0.06001549 ...  0.01716102 -0.01121431
  -0.05491802]
 [ 0.0008452   0.07415237  0.04127556 ...  0.03979664 -0.07446591
   0.00197193]
 ...
 [ 0.0740085  -0.01621768 -0.01278681 ... -0.03371341 -0.01016733
   0.00977206]
 [-0.05664577  0.01628509  0.01583097 ... -0.06301782  0.0059871
   0.09004645]
 [ 0.05109572 -0.02023073  0.007758   ...  0.01385515 -0.0073409
  -0.01043406]]


In [12]:
mess=np.around(message,2)


In [13]:
print(mess)

[[-0.01  0.07  0.07 ...  0.03 -0.01 -0.01]
 [ 0.04  0.02  0.06 ...  0.02 -0.01 -0.05]
 [ 0.    0.07  0.04 ...  0.04 -0.07  0.  ]
 ...
 [ 0.07 -0.02 -0.01 ... -0.03 -0.01  0.01]
 [-0.06  0.02  0.02 ... -0.06  0.01  0.09]
 [ 0.05 -0.02  0.01 ...  0.01 -0.01 -0.01]]


In [14]:
mess=mess*100

In [15]:
em_dict = dict(zip(crop,mess))

In [16]:
import random
import numpy as np
def genRandom(d,end):
    randNum=[]
    for i in range(0,d):
        b=random.randint(0,end)
        randNum.append(b)
    return np.array(randNum)    


In [17]:
s=genRandom(400,999)
print(s)

[504 173 537 799 438 628 304 132  57 796 879 880 669 391 230 514 733 629
 384 708 485 789 288 758 872 402 281 908 116 270 984 466 312 178 680 262
 996 241 406 761 989 869 262 851 622 452 549 549 211 649 553 989  12 857
 133 200 572 189 226 289 256 226 717 266 937 319 334 915 155 388 390 691
 699 918 914 993 338 927 400 527 703 212 448 684  15  34 577 138 789 997
 658 234 327 895 423  89  46 728 650 139 823 854 375 416 199 306 546 788
 737 660 945 909 538 566 901  70 582 441 816 825 693 802 757 455 490 187
 584 269 623 543  45 434 512  46 811  46 629 239   1 325 583 349 783  54
   3 656 235  62 670 539 899 106 963 890 594 699 662 964 676 833 121 679
 709 524 992 385 741 200 814 321  38 221 898 143 609 161 372 431   2  66
 388 732 217 180 369 903  18 952 563 766 776 430 818 966 961 364 619 981
 151 563   3 955 665 973 984  82 371  49 853 123 127 446 986 183 482 154
 356 951 764  88 818 567 384 113 797  29 910 664 671 357 665 303 730 558
 435  71 438 880  54  59 822 951 622 155 648 997 17

In [57]:
from sklearn.model_selection import KFold
from scipy import interpolate
import math
import numpy as np  
  

all_threshold=[]
# LFW functions taken from David Sandberg's FaceNet implementation
def distance(embeddings1, embeddings2, distance_metric=1):
    if distance_metric==0:
        # Euclidian distance
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),1)
    elif distance_metric==1:
        # Distance based on cosine similarity
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=1)
        #norm = np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        #similarity = dot / norm
        similarity=dot
        #dist = np.arccos(similarity) / math.pi
        dist =similarity
    elif distance_metric==2:
        dist=distances(embeddings1,embeddings2)
    else:
        raise 'Undefined distance metric %d' % distance_metric

    return dist

def calculate_roc(thresholds, embeddings1, embeddings2, actual_issame, nrof_folds=10, distance_metric=1, subtract_mean=False):
    assert(embeddings1.shape[0] == embeddings2.shape[0])
    assert(embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=False)

    tprs = np.zeros((nrof_folds,nrof_thresholds))
    fprs = np.zeros((nrof_folds,nrof_thresholds))
    accuracy = np.zeros((nrof_folds))

    is_false_positive = []
    is_false_negative = []

    indices = np.arange(nrof_pairs)

    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        if subtract_mean:
            mean = np.mean(np.concatenate([embeddings1[train_set], embeddings2[train_set]]), axis=0)
        else:
            mean = 0.0
        dist = distance(embeddings1-mean, embeddings2-mean, distance_metric)

        # Find the best threshold for the fold
        acc_train = np.zeros((nrof_thresholds))
        for threshold_idx, threshold in enumerate(thresholds):
            _, _, acc_train[threshold_idx], _ ,_ = calculate_accuracy(threshold, dist[train_set], actual_issame[train_set])
        best_threshold_index = np.argmax(acc_train)
        print(thresholds[best_threshold_index])
        all_threshold.append(thresholds[best_threshold_index])
        for threshold_idx, threshold in enumerate(thresholds):
            tprs[fold_idx,threshold_idx], fprs[fold_idx,threshold_idx], _, _, _ = calculate_accuracy(threshold, dist[test_set], actual_issame[test_set])
        _, _, accuracy[fold_idx], is_fp, is_fn = calculate_accuracy(thresholds[best_threshold_index], dist[test_set], actual_issame[test_set])

        tpr = np.mean(tprs,0)
        fpr = np.mean(fprs,0)
        is_false_positive.extend(is_fp)
        is_false_negative.extend(is_fn)

    return tpr, fpr, accuracy, is_false_positive, is_false_negative

def calculate_accuracy(threshold, dist, actual_issame):
    
    #print(dist)
    #dist=np.around(dist,2)
    #print(dist)
    #threshold=4000
    #predict_issame = np.less(dist, threshold)
    #print(dist)
    
    p=8000
    dimension=dist.shape[0]
    r=genRandom(dimension,99999999)
    dist=dist+r
    dist=dist//p
    
    
    
   

    # 生成维度为4000的ndarray，所有值都是constant_value  
    threshold=2000
    thresh = np.full(dimension, threshold)  
    thresh=thresh+r
    thresh=thresh//p

    rightMax=10000
    
    rightM= np.full(dimension, rightMax)  
    rightM=rightM+r
    rightM=rightM//p
    
    # 打印ndarray的前几个元素以验证  
    #print(thresh[:500])
    
#     dist = [int(num) for num in dist]
#     #print(dist)
#     threshold=40
    
    print(type(dist))
    
    
   
    #print(len(dist))
    # 组合两个条件：dist > thresh 且 dist <= rightM
    predict_issame = (dist > thresh) & (dist <= rightM)
    
    
    
    #predict_issame = np.greater(dist, thresh)
    #predict_issame = np.greater(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))

    is_fp = np.logical_and(predict_issame, np.logical_not(actual_issame))
    is_fn = np.logical_and(np.logical_not(predict_issame), actual_issame)

    tpr = 0 if (tp+fn==0) else float(tp) / float(tp+fn)
    fpr = 0 if (fp+tn==0) else float(fp) / float(fp+tn)
    acc = float(tp+tn)/dist.size
    #acc = float(tp+tn)/len(dist)
    return tpr, fpr, acc, is_fp, is_fn

def calculate_val(thresholds, embeddings1, embeddings2, actual_issame, far_target, nrof_folds=10, distance_metric=1, subtract_mean=False):
    assert(embeddings1.shape[0] == embeddings2.shape[0])
    assert(embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=False)

    val = np.zeros(nrof_folds)
    far = np.zeros(nrof_folds)

    indices = np.arange(nrof_pairs)

    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        if subtract_mean:
            mean = np.mean(np.concatenate([embeddings1[train_set], embeddings2[train_set]]), axis=0)
        else:
            mean = 0.0
        #mean=0.0
        dist = distance(embeddings1-mean, embeddings2-mean, distance_metric)

        #Find the threshold that gives FAR = far_target
        far_train = np.zeros(nrof_thresholds)
        for threshold_idx, threshold in enumerate(thresholds):
            _, far_train[threshold_idx] = calculate_val_far(threshold, dist[train_set], actual_issame[train_set])
        if np.max(far_train)>=far_target:
            f = interpolate.interp1d(far_train, thresholds, kind='slinear')
            threshold = f(far_target)
        else:
            threshold = 0.0
       # threshold=4000
     

        val[fold_idx], far[fold_idx] = calculate_val_far(threshold, dist[test_set], actual_issame[test_set])

    val_mean = np.mean(val)
    far_mean = np.mean(far)
    val_std = np.std(val)
    return val_mean, val_std, far_mean

def calculate_val_far(threshold, dist, actual_issame):
    #threshold=0.41
    predict_issame = np.greater(dist, threshold)
    true_accept = np.sum(np.logical_and(predict_issame, actual_issame))
    false_accept = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    n_same = np.sum(actual_issame)
    n_diff = np.sum(np.logical_not(actual_issame))
    val = float(true_accept) / float(n_same)
    far = float(false_accept) / float(n_diff)
    return val, far



def evaluate(embeddings, actual_issame, nrof_folds=10, distance_metric=1, subtract_mean=False):
    # Calculate evaluation metrics
    print(embeddings.shape)
    #thresholds = np.arange(-1, 1, 0.01) 
    thresholds = np.arange(-10000, 10000, 100) 
    #thresholds = np.arange(-40000, 40000, 100) 
    embeddings1 = embeddings[0::2]
    embeddings2 = embeddings[1::2]
    tpr, fpr, accuracy, fp, fn  = calculate_roc(thresholds, embeddings1, embeddings2,
        np.asarray(actual_issame), nrof_folds=nrof_folds, distance_metric=distance_metric, subtract_mean=subtract_mean)
    thresholds = np.arange(-10000, 10000, 10)
    #thresholds = np.arange(-1, 1, 0.001)
    #thresholds = np.arange(-40000, 40000,10)
    val, val_std, far = calculate_val(thresholds, embeddings1, embeddings2,
        np.asarray(actual_issame), 1e-2, nrof_folds=nrof_folds, distance_metric=distance_metric, subtract_mean=subtract_mean)
    return tpr, fpr, accuracy, val, val_std, far, fp, fn

def add_extension(path):
#     if os.path.exists(path+'.jpg'):
#         return path+'.jpg'
#     elif os.path.exists(path+'.png'):
#         return path+'.png'
#     else:
#         raise RuntimeError('No file "%s" with extension png or jpg.' % path)
    return path+'.jpg'

def get_paths(lfw_dir, pairs):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    for pair in pairs:
        if len(pair) == 3:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[2])))
            issame = True
        elif len(pair) == 4:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[2], pair[2] + '_' + '%04d' % int(pair[3])))
            issame = False
        path_list += (path0,path1)
        issame_list.append(issame)
#         if os.path.exists(path0) and os.path.exists(path1):    # Only add the pair if both paths exist
#             path_list += (path0,path1)
#             issame_list.append(issame)
#         else:
#             nrof_skipped_pairs += 1
    if nrof_skipped_pairs>0:
        print('Skipped %d image pairs' % nrof_skipped_pairs)

    return path_list, issame_list

def read_pairs(pairs_filename):
    pairs = []
    with open(pairs_filename, 'r') as f:
        for line in f.readlines()[1:]:
            pair = line.strip().split()
            pairs.append(pair)
    return np.array(pairs, dtype=object)

In [56]:
# import numpy as np  
  
# # 定义常数  
# constant_value = 4000  # 你可以将这里的值改为任何你需要的常数  
  
# # 生成维度为4000的ndarray，所有值都是constant_value  
# ndarray = np.full(4000, constant_value)  
  
# # 打印ndarray的前几个元素以验证  
# print(ndarray[:500])

In [40]:
#mess=np.around(message,8)
# mess=message

# 

In [74]:
# mess=np.around(message,8)

In [75]:
# em_dict = dict(zip(crop,mess))

In [58]:
pairs = read_pairs(pairs_path)
path_list, issame_list = get_paths(data_dir, pairs)
embeddings = np.array([em_dict[path] for path in path_list])

tpr, fpr, accuracy, val, val_std, far, fp, fn = evaluate(embeddings, issame_list)

(12000, 512)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<cl

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [59]:
print(accuracy)
np.mean(accuracy)





[0.69       0.71       0.7        0.72       0.695      0.665
 0.67833333 0.685      0.695      0.67333333]


0.6911666666666667

In [41]:
np.mean(all_threshold)


-7940.0

In [22]:
print(len(pairs))

6000


In [1025]:
    #a little test for look the error situation
    dist = np.array([2100,2200.2299,2500, 3000, 4000,5000,9000,10000])
    p=1000
    dimension=dist.shape[0]
    r=genRandom(dimension,99999999)
    print(r)
    dist=dist+r
    dist=dist//p
    print(dist)

    # 生成维度为4000的ndarray，所有值都是constant_value  
    threshold=2300
    thresh = np.full(dimension, threshold)  
    thresh=thresh+r
    thresh=thresh//p
    print(thresh)

    rightMax=10000
    rightM= np.full(dimension, rightMax)  
    rightM=rightM+r
    rightM=rightM//p
    print(rightM)

    # 组合两个条件：dist > thresh 且 dist <= rightM
    predict_issame = (dist > thresh) & (dist <= rightM)
    print(predict_issame)

[55633634 45308601 12542928 70346752  9871909 11995441 77442695 64250201]
[55635. 45310. 12545. 70349.  9875. 12000. 77451. 64260.]
[55635 45310 12545 70349  9874 11997 77444 64252]
[55643 45318 12552 70356  9881 12005 77452 64260]
[False False False False  True  True  True  True]
